# Postgres CRUD handler

In [ ]:
from modules.postgres_crud.postgres_crud import PostgresCrud as PG
from datetime import datetime

In [ ]:
DB_HOST='localhost'
POSTGRES_DB='pg_dev'
POSTGRES_USER='openbot'
POSTGRES_PASSWORD='06Fj@%r7KTXm5+eWn2'

In [ ]:
pg = PG(DB_HOST, POSTGRES_DB, POSTGRES_USER, POSTGRES_PASSWORD)

In [1]:
# auxiliary_functions.py

def utc_time_func():

    try:
    
        url = 'http://worldclockapi.com/api/json/utc/now'

        time_utc_now = requests.get(url).json

        year = int(time_utc_now()['currentDateTime'].split('T')[0].split('-')[0])
        month = int(time_utc_now()['currentDateTime'].split('T')[0].split('-')[1])
        day = int(time_utc_now()['currentDateTime'].split('T')[0].split('-')[2])

        hour = int(time_utc_now()['currentDateTime'].split('T')[1].split(':')[0])
        minute = int(time_utc_now()['currentDateTime'].split('T')[1].split(':')[1].split('Z')[0])
    
        utc_now = datetime(year, month, day, hour, minute)
    
    except (Exception) as error: #TODO: Tratar exceção
        
        utc_now = datetime.utcnow()

    return utc_now

def delta_time_in_seconds_between_utc_and(this_time):

    utc_time = utc_time_func()
    
    delta_time = utc_time - this_time

    delta_hour = round(delta_time.total_seconds()/3600)
    
    delta = delta_hour*3600
    
    return delta

In [2]:
#binance_handler.py

import requests
import requests
import json
import time
from datetime import datetime
from copy import deepcopy
#import delta_time_in_seconds_between_utc_and

def server_time():

    binance_time = datetime.fromtimestamp(int((requests.get('https://api.binance.com/api/v1/time').\
                                               json()['serverTime'])/1000))

    return binance_time


class GetKlines:
    
    def __init__(self, asset_symbol, candle_interval, start_time, max_attempts):
        
        self.asset_symbol = asset_symbol
        self.candle_interval = candle_interval
        self.start_time = start_time
        self.max_attempts = max_attempts
  
    
    def formated(self):
        
        klines = []
        
        for i in range(5):

            raw_klines = self.raw()

            if(raw_klines): klines = self._format(raw_klines); break

            else: time.sleep(2)
        
        return klines
    
    
    def raw(self):
        
        raw_klines = []

        url = '''https://api.binance.com/api/v1/klines?symbol=\
''' + self.asset_symbol + '''&interval=''' + self.candle_interval +\
'''&startTime=''' + self.start_time

        for i in range(max_attempts):

            try:

                response = requests.get(url)

                response.raise_for_status()

            except (Exception, requests.exceptions.RequestException) as error: #TODO: TRATAR EXCEÇÃO AQUI

                time.sleep(5)

            finally:

                if (int(response.status_code) == 200): raw_klines = response.json(); break
            
        return raw_klines
    
    def is_there_missing_data_in(klines_in):
        pass
    
    
    def _format(self, klines_in):
        
        def completing_missing_data_on_1_minute_candle():
            pass

        def clear_and_adjust_time_prices_and_volume_of(klines_in):

            klines_out = []

            delta = delta_time_in_seconds_between_utc_and(server_time())
            
            for i in range (len(klines_in)):

                open_time = datetime.fromtimestamp(int(klines_in[i][0]/1000) + delta)
                
                volume =  (float(klines_in[i][5]))

                data = [open_time,
                        float(klines_in[i][1]), #Open
                        float(klines_in[i][2]), #High
                        float(klines_in[i][3]), #Low
                        float(klines_in[i][4]), #Close
                        volume]

                klines_out.append(data)

            return klines_out

        def making_seconds_be_zero_on(klines_in):

            klines_out = deepcopy(klines_in)

            for i in range(len(klines_in)):

                entry = klines_in[i][0]

                if (entry.second != 0):

                    out = datetime.fromtimestamp((datetime.timestamp(entry) - entry.second))

                    klines_out[i][0] = out

            return klines_out
        
        klines = clear_and_adjust_time_prices_and_volume_of(klines_in)
        
        klines = making_seconds_be_zero_on(klines)
        
        return klines

# Testing binance klines

In [3]:
asset_symbol = 'BTCUSDT'
candle_interval = '1m'
start_time = '1241893500000'
max_attempts = 10

get_binance_klines = GetKlines(asset_symbol ,candle_interval, start_time, max_attempts)

In [4]:
original_klines = get_binance_klines.raw()

In [10]:
formated_klines = get_binance_klines.formated()

In [12]:
original_klines

[[1502942400000,
  '4261.48000000',
  '4261.48000000',
  '4261.48000000',
  '4261.48000000',
  '1.77518300',
  1502942459999,
  '7564.90685084',
  3,
  '0.07518300',
  '320.39085084',
  '7960.54017996'],
 [1502942460000,
  '4261.48000000',
  '4261.48000000',
  '4261.48000000',
  '4261.48000000',
  '0.00000000',
  1502942519999,
  '0.00000000',
  0,
  '0.00000000',
  '0.00000000',
  '7960.45262989'],
 [1502942520000,
  '4280.56000000',
  '4280.56000000',
  '4280.56000000',
  '4280.56000000',
  '0.26107400',
  1502942579999,
  '1117.54292144',
  2,
  '0.26107400',
  '1117.54292144',
  '7960.37529526'],
 [1502942580000,
  '4261.48000000',
  '4261.48000000',
  '4261.48000000',
  '4261.48000000',
  '0.01200800',
  1502942639999,
  '51.17185184',
  3,
  '0.01200800',
  '51.17185184',
  '7960.37529526'],
 [1502942640000,
  '4261.48000000',
  '4261.48000000',
  '4261.48000000',
  '4261.48000000',
  '0.14079600',
  1502942699999,
  '599.99933808',
  1,
  '0.14079600',
  '599.99933808',
  '7960.

In [11]:
formated_klines

[[datetime.datetime(2017, 8, 17, 4, 0),
  4261.48,
  4261.48,
  4261.48,
  4261.48,
  1.775183],
 [datetime.datetime(2017, 8, 17, 4, 1),
  4261.48,
  4261.48,
  4261.48,
  4261.48,
  0.0],
 [datetime.datetime(2017, 8, 17, 4, 2),
  4280.56,
  4280.56,
  4280.56,
  4280.56,
  0.261074],
 [datetime.datetime(2017, 8, 17, 4, 3),
  4261.48,
  4261.48,
  4261.48,
  4261.48,
  0.012008],
 [datetime.datetime(2017, 8, 17, 4, 4),
  4261.48,
  4261.48,
  4261.48,
  4261.48,
  0.140796],
 [datetime.datetime(2017, 8, 17, 4, 5),
  4261.48,
  4261.48,
  4261.48,
  4261.48,
  0.0],
 [datetime.datetime(2017, 8, 17, 4, 6),
  4261.48,
  4261.48,
  4261.48,
  4261.48,
  0.0],
 [datetime.datetime(2017, 8, 17, 4, 7),
  4261.48,
  4261.48,
  4261.48,
  4261.48,
  0.0],
 [datetime.datetime(2017, 8, 17, 4, 8),
  4261.48,
  4261.48,
  4261.48,
  4261.48,
  0.0],
 [datetime.datetime(2017, 8, 17, 4, 9),
  4261.48,
  4261.48,
  4261.48,
  4261.48,
  0.0],
 [datetime.datetime(2017, 8, 17, 4, 10),
  4261.48,
  4261.4

In [ ]:
#DRAFT: criando lista auxiliar de open_times

open_time_list_in = []

for i in range(len(klines) - 1):
    
    open_time_entry = klines[i][0]
    
    #print (open_time_entry)
    
    open_time_list_in.append(open_time_entry)

In [ ]:
def aux_open_time(data_in):
    
    aux_open_time_list = []
    
    current_open_time = data_in.loc[0, 'open_time']

    last_open_time = data_in.loc[len(data_in)-1, 'open_time']
    
    while (current_open_time < last_open_time):
        
        aux_open_time_list.append(current_open_time)
        
        current_open_time = datetime.fromtimestamp(datetime.timestamp(current_open_time) + 60)
        
    return aux_open_time_list

In [ ]:
#Para anexar aos testes

from copy import deepcopy

def baguncar_klines(klines_in):
    
    klines_out = deepcopy(klines_in)
    
    for i in range(len(klines_in)):
      
        klines_out[i][0] = (klines_in[i][0] + 40000)
    
    return klines_out

In [ ]:
#Trash

